In [2]:
import pandas as pd 
import numpy as np 
import time 

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier,BaggingClassifier,ExtraTreesClassifier

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import f1_score

from sklearn.decomposition import PCA

In [3]:
mat =np.asarray([]) #emo
mat1 =np.asarray([]) #senti
data = pd.read_csv('audio_features.csv')
data = np.asarray(data)

In [4]:
output = data[:,4:6] #emotions #sentiments

In [5]:
inp = data[:,13:] #input-features

# Train_Test 

In [6]:
X_train, X_test, y_train, y_test = train_test_split(inp, output, test_size=0.2, shuffle = True)

scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

X_train_copy1 = X_train.copy()
X_test_copy1 = X_test.copy()

# emotion (7)
y_train_emo = y_train[:,0] *1.0
y_train_emo = y_train_emo.astype('float')
y_test_emo = y_test[:,0] *1.0
y_test_emo = y_test_emo.astype('float')

# sentiment(3)
y_train_senti = y_train[:,1] *1.0
y_train_senti = y_train_senti.astype('float')
y_test_senti = y_test[:,1] *1.0
y_test_senti = y_test_senti.astype('float')

# ------------------------------------------------------------------------------------------------------

# Multinomial Logistic Regression

#1. Lot of Feature Engineering is required.

#2. If the independent features are correlated with each other it may affect the performance of the classifier.

#3. It is quite sensitive to noise and overfitting.

In [6]:
def logistic_regression(X_train, y_train, X_test, y_test):
    print("Multinomial Logistic Regression")
    start = time.time()
    logreg = LogisticRegression(penalty='l2',multi_class='multinomial', max_iter = 1e4, solver = 'lbfgs')
    #penalty to handle size better
    
    #https://towardsdatascience.com/dont-sweat-the-solver-stuff-aea7cddc3451
    
    logreg.fit(X_train, y_train)
    accuracy_train_data = cross_val_score(logreg, X_train,y_train,cv=5)
    print("Training Accuracy : ",np.mean(accuracy_train_data))
    accuracy_test_data = cross_val_score(logreg, X_test,y_test,cv=5)
    print("Testing Accuracy : ",np.mean(accuracy_test_data))
    y_pred = logreg.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    print(cm)
    f1 = f1_score(y_test, y_pred, average='micro')
    print("F1-Score : ",f1)
    print("Time Taken :", (time.time()-start))
    return np.mean(accuracy_train_data), np.std(accuracy_train_data), np.mean(accuracy_test_data), np.std(accuracy_test_data), f1, time.time()-start

print("FOR EMOTIONS (7): ")
tup = logistic_regression(X_train, y_train_emo, X_test, y_test_emo)
d = np.asarray(tup)
mat = np.append(mat,d)
print("\nFOR SENTIMENTS (3): ")
tup = logistic_regression(X_train, y_train_senti, X_test, y_test_senti)
d = np.asarray(tup)
mat1 = np.append(mat1,d)

FOR EMOTIONS (7): 
Multinomial Logistic Regression
Training Accuracy :  0.4264081329464223
Testing Accuracy :  0.383663728344834
[[926  44  11  47 128  11 131]
 [136  53   4  10  47   2  68]
 [ 50   3   1   1   7   0  17]
 [102   9   4  20  27   2  28]
 [238  21   6  17  90   5  80]
 [ 31   7   0   4  14   4  22]
 [124  12   6  10  51   3 108]]
F1-Score :  0.43836615609044494
Time Taken : 283.76397490501404

FOR SENTIMENTS (3): 
Multinomial Logistic Regression
Training Accuracy :  0.49516544104657195
Testing Accuracy :  0.43799675587996756
[[883 140 275]
 [283 151 167]
 [332 133 378]]
F1-Score :  0.5149525893508388
Time Taken : 132.4779531955719


# Naive Bayes

In [7]:
def naive_bayes(X_train, y_train, X_test, y_test):
    start = time.time()    
    print("Multinomial Naive Bayes")
    classifier = MultinomialNB();
    classifier.fit(X_train, y_train)
    accuracy_train_data = cross_val_score(classifier, X_train,y_train,cv=5)
    print("Training Accuracy : ",np.mean(accuracy_train_data))
    
    # predicting test set results
    y_pred = classifier.predict(X_test)
    accuracy_test_data = cross_val_score(classifier, X_test,y_test,cv=5)
    print("Testing Accuracy : ",np.mean(accuracy_test_data))
    
    # making the confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    print(cm)
    f1 = f1_score(y_test, y_pred, average='micro')
    print("F1-Score : ",f1)
    print("Time Taken :", (time.time()-start))
    return np.mean(accuracy_train_data), np.std(accuracy_train_data), np.mean(accuracy_test_data), np.std(accuracy_test_data), f1, time.time()-start


print("FOR EMOTIONS (7): ")
tup = naive_bayes(X_train, y_train_emo, X_test, y_test_emo)
d = np.asarray(tup)
mat = np.append(mat,d)
print("\nFOR SENTIMENTS (3): ")
tup = naive_bayes(X_train, y_train_senti, X_test, y_test_senti)
d = np.asarray(tup)
mat1 = np.append(mat1,d)

FOR EMOTIONS (7): 
Multinomial Naive Bayes
Training Accuracy :  0.31889587362790617
Testing Accuracy :  0.32969167564117907
[[541 256   0 321  19   8 153]
 [102 102   0  50  15   3  48]
 [ 35  13   0  13   2   1  15]
 [ 64  16   0  70   4   1  37]
 [157 104   0  91  16   6  83]
 [ 21  12   0  29   4   1  15]
 [ 77  47   0  63  11   3 113]]
F1-Score :  0.3074398249452954
Time Taken : 0.4596216678619385

FOR SENTIMENTS (3): 
Multinomial Naive Bayes
Training Accuracy :  0.43461552690440824
Testing Accuracy :  0.4409277651469826
[[549 192 557]
 [207 139 255]
 [244 146 453]]
F1-Score :  0.4161196207148067
Time Taken : 0.4506723880767822


# Random Forest

In [8]:
def random_forest(X_train, y_train, X_test, y_test):
    start = time.time()
    print("Random Forest Classifier")
    rf = RandomForestClassifier(random_state=42)
    rf.fit(X_train, y_train)
    accuracy_train_data = cross_val_score(rf, X_train,y_train,cv=5)
    print("Training Accuracy : ",np.mean(accuracy_train_data))

    # predicting test set results
    y_pred = rf.predict(X_test)
    accuracy_test_data = cross_val_score(rf, X_test,y_test,cv=5)
    print("Testing Accuracy : ",np.mean(accuracy_test_data))
    # making the confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    print(cm)
    f1 = f1_score(y_test, y_pred, average='micro')
    print("F1-Score : ",f1)
    print("Time Taken :", (time.time()-start))
    return np.mean(accuracy_train_data), np.std(accuracy_train_data), np.mean(accuracy_test_data), np.std(accuracy_test_data), f1, time.time()-start


print("FOR EMOTIONS (7): ")
tup = random_forest(X_train, y_train_emo, X_test, y_test_emo)
d = np.asarray(tup)
mat = np.append(mat,d)
print("\nFOR SENTIMENTS (3): ")
tup= random_forest(X_train, y_train_senti, X_test, y_test_senti)
d = np.asarray(tup)
mat1 = np.append(mat1,d)

FOR EMOTIONS (7): 
Random Forest Classifier
Training Accuracy :  0.47546980718046683
Testing Accuracy :  0.4821274247802906
[[1244    6    0    1   23    0   24]
 [ 282    4    0    0   16    0   18]
 [  72    0    0    0    2    0    5]
 [ 171    1    0    0    5    0   15]
 [ 399    6    0    0   29    0   23]
 [  72    0    0    0    2    0    8]
 [ 238    5    0    1   13    0   57]]
F1-Score :  0.4865061998541211
Time Taken : 163.97067308425903

FOR SENTIMENTS (3): 
Random Forest Classifier
Training Accuracy :  0.5065658070906809
Testing Accuracy :  0.5149482137396462
[[1066   32  200]
 [ 384   49  168]
 [ 505   34  304]]
F1-Score :  0.5175054704595186
Time Taken : 140.2228865623474


# Support Vector Classifier

In [9]:
def sv_classifier(X_train, y_train, X_test, y_test):
    start = time.time()
    print("Support Vector Classifier")
    clf = make_pipeline(StandardScaler(with_mean=False), SVC(gamma='auto'))
    clf.fit(X_train, y_train)
    accuracy_train_data = cross_val_score(clf, X_train,y_train,cv=5)
    print("Training Accuracy : ",np.mean(accuracy_train_data))
    
    # predicting test set results
    y_pred = clf.predict(X_test)
    accuracy_test_data = cross_val_score(clf, X_test,y_test,cv=5)
    print("Testing Accuracy : ",np.mean(accuracy_test_data))
    # making the confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='micro')
    print("F1-Score : ",f1)
    print(cm)
    print("Time Taken :", (time.time()-start))
    return np.mean(accuracy_train_data), np.std(accuracy_train_data), np.mean(accuracy_test_data), np.std(accuracy_test_data), f1, time.time()-start

print("FOR EMOTIONS (7): ")
tup = sv_classifier(X_train, y_train_emo, X_test, y_test_emo)
d = np.asarray(tup)
mat = np.append(mat,d)
print("\nFOR SENTIMENTS (3): ")
tup = sv_classifier(X_train, y_train_senti, X_test, y_test_senti)
d = np.asarray(tup)
mat1 = np.append(mat1,d)

FOR EMOTIONS (7): 
Support Vector Classifier
Training Accuracy :  0.4798469149165676
Testing Accuracy :  0.4781161501336205
F1-Score :  0.49270605397520056
[[1259    1    0    0    8    0   30]
 [ 281    7    0    0   12    0   20]
 [  68    2    0    0    0    0    9]
 [ 174    1    0    0    2    0   15]
 [ 408    4    0    0   18    0   27]
 [  67    1    0    0    1    0   13]
 [ 234    3    0    0   10    0   67]]
Time Taken : 750.3730466365814

FOR SENTIMENTS (3): 
Support Vector Classifier
Training Accuracy :  0.5333752334539209
Testing Accuracy :  0.5226097882015077
F1-Score :  0.5408460977388767
[[1051   27  220]
 [ 377   58  166]
 [ 430   39  374]]
Time Taken : 530.1335372924805


In [15]:
X_train, X_test, y_train, y_test = train_test_split(inp, output, test_size=0.2, shuffle = True)

X_train_copy1 = X_train.copy()
X_test_copy1 = X_test.copy()

# emotion (7)
y_train_emo = y_train[:,0] *1.0
y_train_emo = y_train_emo.astype('float')
y_test_emo = y_test[:,0] *1.0
y_test_emo = y_test_emo.astype('float')

# sentiment(3)
y_train_senti = y_train[:,1] *1.0
y_train_senti = y_train_senti.astype('float')
y_test_senti = y_test[:,1] *1.0
y_test_senti = y_test_senti.astype('float')

In [16]:
def sv_classifier(X_train, y_train, X_test, y_test):
    start = time.time()
    print("Support Vector Classifier")
    clf = make_pipeline(StandardScaler(with_mean=False), SVC(gamma='auto'))
    clf.fit(X_train, y_train)
    accuracy_train_data = cross_val_score(clf, X_train,y_train,cv=5)
    print("Training Accuracy : ",np.mean(accuracy_train_data))
    
    # predicting test set results
    y_pred = clf.predict(X_test)
    accuracy_test_data = cross_val_score(clf, X_test,y_test,cv=5)
    print("Testing Accuracy : ",np.mean(accuracy_test_data))
    # making the confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='micro')
    print("F1-Score : ",f1)
    print(cm)
    print("Time Taken :", (time.time()-start))
    return np.mean(accuracy_train_data), np.std(accuracy_train_data), np.mean(accuracy_test_data), np.std(accuracy_test_data), f1, time.time()-start

print("FOR EMOTIONS (7): ")
tup = sv_classifier(X_train, y_train_emo, X_test, y_test_emo)
d = np.asarray(tup)
mat = np.append(mat,d)
print("\nFOR SENTIMENTS (3): ")
tup = sv_classifier(X_train, y_train_senti, X_test, y_test_senti)
d = np.asarray(tup)
mat1 = np.append(mat1,d)

FOR EMOTIONS (7): 
Support Vector Classifier
Training Accuracy :  0.4842246045987876
Testing Accuracy :  0.4762946565088482
F1-Score :  0.4883296863603209
[[1273    4    0    0    6    0   22]
 [ 279   10    0    0    6    0   14]
 [  51    1    0    0    1    0   10]
 [ 198    1    0    0    1    0   10]
 [ 445    4    0    0   15    0   19]
 [  73    1    0    0    0    0    3]
 [ 245    1    0    0    8    0   41]]
Time Taken : 751.3273503780365

FOR SENTIMENTS (3): 
Support Vector Classifier
Training Accuracy :  0.5345621541317551
Testing Accuracy :  0.5123967931075745
F1-Score :  0.5320933625091174
[[1139   25  141]
 [ 445   45  124]
 [ 514   34  275]]
Time Taken : 564.9820585250854


# ----------------------------------------------------------------------------------------------------------

# Feature Engineering - 1. PCA: Dimensionality Reduction

In [7]:
pca = PCA(n_components=300, svd_solver='full')
X_train1 = pca.fit_transform(X_train.copy())
# We keep only 20% data 
print(sum(pca.explained_variance_ratio_))
# 83.3% info retained

X_test1 = pca.transform(X_test.copy())

X_train1 = scaler.fit_transform(X_train1)
X_test1 = scaler.fit_transform(X_test1)

0.846261122193179


# Multinomial Logistic Regression

In [9]:
def logistic_regression(X_train, y_train, X_test, y_test):
    print("Multinomial Logistic Regression")
    start = time.time()
    logreg = LogisticRegression(penalty='l2',multi_class='multinomial', max_iter = 1e4, solver = 'lbfgs')
    #penalty to handle size better
    
    #https://towardsdatascience.com/dont-sweat-the-solver-stuff-aea7cddc3451
    
    logreg.fit(X_train, y_train)
    accuracy_train_data = cross_val_score(logreg, X_train,y_train,cv=5)
    print("Training Accuracy : ",np.mean(accuracy_train_data))
    accuracy_test_data = cross_val_score(logreg, X_test,y_test,cv=5)
    print("Testing Accuracy : ",np.mean(accuracy_test_data))
    y_pred = logreg.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    print(cm)
    f1 = f1_score(y_test, y_pred, average='micro')
    print("F1-Score : ",f1)
    print("Time Taken :", (time.time()-start))


print("FOR EMOTIONS (7): ")
tup = logistic_regression(X_train1, y_train_emo, X_test1, y_test_emo)
d = np.asarray(tup)
mat = np.append(mat,d)
print("\nFOR SENTIMENTS (3): ")
tup = logistic_regression(X_train1, y_train_senti, X_test1, y_test_senti)
d = np.asarray(tup)
mat1 = np.append(mat1,d)

FOR EMOTIONS (7): 
Multinomial Logistic Regression
Training Accuracy :  0.46662360265384056
Testing Accuracy :  0.42924427957932804
[[1165   59    2    5   30    1   40]
 [ 225   53    0    2   12    0   25]
 [  51    3    0    1    1    0    6]
 [ 158   18    2    9    0    0   13]
 [ 364   28    0    0   34    1   23]
 [  67    2    0    0    5    1    8]
 [ 218   30    2    1   19    0   58]]
F1-Score :  0.4814004376367615
Time Taken : 31.644580602645874

FOR SENTIMENTS (3): 
Multinomial Logistic Regression
Training Accuracy :  0.5072945283347489
Testing Accuracy :  0.4609768258146863
[[1074   48  180]
 [ 408   66  132]
 [ 502   43  289]]
F1-Score :  0.5211524434719184
Time Taken : 6.857990741729736


# Naive Bayes

In [10]:
def naive_bayes(X_train, y_train, X_test, y_test):
    start = time.time()    
    print("Multinomial Naive Bayes")
    classifier = MultinomialNB();
    classifier.fit(X_train, y_train)
    accuracy_train_data = cross_val_score(classifier, X_train,y_train,cv=5)
    print("Training Accuracy : ",np.mean(accuracy_train_data))
    
    # predicting test set results
    y_pred = classifier.predict(X_test)
    accuracy_test_data = cross_val_score(classifier, X_test,y_test,cv=5)
    print("Testing Accuracy : ",np.mean(accuracy_test_data))
    
    # making the confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    print(cm)
    f1 = f1_score(y_test, y_pred, average='micro')
    print("F1-Score : ",f1)
    print("Time Taken :", (time.time()-start))
    return np.mean(accuracy_train_data), np.std(accuracy_train_data), np.mean(accuracy_test_data), np.std(accuracy_test_data), f1, time.time()-start

print("FOR EMOTIONS (7): ")
tup = naive_bayes(X_train1, y_train_emo, X_test1, y_test_emo)
d = np.asarray(tup)
mat = np.append(mat,d)

print("\nFOR SENTIMENTS (3): ")
tup = naive_bayes(X_train1, y_train_senti, X_test1, y_test_senti)
d = np.asarray(tup)
mat1 = np.append(mat1,d)

FOR EMOTIONS (7): 
Multinomial Naive Bayes
Training Accuracy :  0.46817440592653925
Testing Accuracy :  0.4748367968303352
[[1302    0    0    0    0    0    0]
 [ 317    0    0    0    0    0    0]
 [  62    0    0    0    0    0    0]
 [ 200    0    0    0    0    0    0]
 [ 450    0    0    0    0    0    0]
 [  83    0    0    0    0    0    0]
 [ 328    0    0    0    0    0    0]]
F1-Score :  0.474835886214442
Time Taken : 0.15608930587768555

FOR SENTIMENTS (3): 
Multinomial Naive Bayes
Training Accuracy :  0.4681743643589593
Testing Accuracy :  0.47483613205164
[[1302    0    0]
 [ 606    0    0]
 [ 834    0    0]]
F1-Score :  0.474835886214442
Time Taken : 0.14336061477661133


# Random Forest

In [11]:
def random_forest(X_train, y_train, X_test, y_test):
    start = time.time()
    print("Random Forest Classifier")
    rf = RandomForestClassifier(random_state=42)
    rf.fit(X_train, y_train)
    accuracy_train_data = cross_val_score(rf, X_train,y_train,cv=5)
    print("Training Accuracy : ",np.mean(accuracy_train_data))

    # predicting test set results
    y_pred = rf.predict(X_test)
    accuracy_test_data = cross_val_score(rf, X_test,y_test,cv=5)
    print("Testing Accuracy : ",np.mean(accuracy_test_data))
    # making the confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    print(cm)
    f1 = f1_score(y_test, y_pred, average='micro')
    print("F1-Score : ",f1)
    print("Time Taken :", (time.time()-start))
    return np.mean(accuracy_train_data), np.std(accuracy_train_data), np.mean(accuracy_test_data), np.std(accuracy_test_data), f1, time.time()-start


print("FOR EMOTIONS (7): ")
tup = random_forest(X_train1, y_train_emo, X_test1, y_test_emo)
d = np.asarray(tup)
mat = np.append(mat,d)
print("\nFOR SENTIMENTS (3): ")
tup= random_forest(X_train1, y_train_senti, X_test1, y_test_senti)
d = np.asarray(tup)
mat1 = np.append(mat1,d)

FOR EMOTIONS (7): 
Random Forest Classifier
Training Accuracy :  0.46835672133219103
Testing Accuracy :  0.4741081993804263
[[1287    3    0    0   11    0    1]
 [ 308    0    0    0    9    0    0]
 [  61    0    0    0    1    0    0]
 [ 199    1    0    0    0    0    0]
 [ 440    2    0    0    8    0    0]
 [  81    1    0    0    1    0    0]
 [ 316    0    0    0   10    0    2]]
F1-Score :  0.47301239970824216
Time Taken : 83.2385904788971

FOR SENTIMENTS (3): 
Random Forest Classifier
Training Accuracy :  0.47209568358093057
Testing Accuracy :  0.48067621288872936
[[1134   21  147]
 [ 492   22   92]
 [ 667   14  153]]
F1-Score :  0.4773887673231218
Time Taken : 69.03017592430115


# Support Vector Machines

In [12]:
def sv_classifier(X_train, y_train, X_test, y_test):
    start = time.time()
    print("Support Vector Classifier")
    clf = make_pipeline(StandardScaler(with_mean=False), SVC(gamma='auto'))
    clf.fit(X_train, y_train)
    accuracy_train_data = cross_val_score(clf, X_train,y_train,cv=5)
    print("Training Accuracy : ",np.mean(accuracy_train_data))
    
    # predicting test set results
    y_pred = clf.predict(X_test)
    accuracy_test_data = cross_val_score(clf, X_test,y_test,cv=5)
    print("Testing Accuracy : ",np.mean(accuracy_test_data))
    # making the confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='micro')
    print("F1-Score : ",f1)
    print(cm)
    print("Time Taken :", (time.time()-start))
    return np.mean(accuracy_train_data), np.std(accuracy_train_data), np.mean(accuracy_test_data), np.std(accuracy_test_data), f1, time.time()-start


print("FOR EMOTIONS (7): ")
tup = sv_classifier(X_train1, y_train_emo, X_test1, y_test_emo)
d = np.asarray(tup)
mat = np.append(mat,d)
print("\nFOR SENTIMENTS (3): ")
tup = sv_classifier(X_train1, y_train_senti, X_test1, y_test_senti)
d = np.asarray(tup)
mat1 = np.append(mat1,d)

FOR EMOTIONS (7): 
Support Vector Classifier
Training Accuracy :  0.4706366199571771
Testing Accuracy :  0.47447249810538067
F1-Score :  0.4759299781181619
[[1301    0    0    0    1    0    0]
 [ 317    0    0    0    0    0    0]
 [  62    0    0    0    0    0    0]
 [ 199    0    0    0    1    0    0]
 [ 448    0    0    0    1    0    1]
 [  83    0    0    0    0    0    0]
 [ 325    0    0    0    0    0    3]]
Time Taken : 157.43177819252014

FOR SENTIMENTS (3): 
Support Vector Classifier
Training Accuracy :  0.5113996178276701
Testing Accuracy :  0.49963104782417933
F1-Score :  0.5269876002917578
[[1226    7   69]
 [ 501   26   79]
 [ 630   11  193]]
Time Taken : 108.41364884376526
